In [1]:
import sys
sys.path.append('/Users/blais.ch/Documents/_01_code/RMG_env_1/RMG-Py/')
import os
import shutil
import sys
import itertools

from rmgpy.chemkin import load_chemkin_file
from rmgpy.data.surface import MetalPropertyLibrary, SitePropertyLibrary, MetalPropertyDatabase, SitePropertyDatabase
from rmgpy.data.thermo import ThermoDatabase

## verify that the metalproperties databases load correctly

In [2]:
mpdb = MetalPropertyDatabase()
spdb = SitePropertyDatabase()
path = '/Users/blais.ch/Documents/_01_code/RMG_env_1/RMG-database/input/surface/'
mpdb.load(path)
spdb.load(path)

## check combinatorial number of sites possible

In [3]:
coords_111 = spdb.get_all_coordination_numbers_on_facet('111')
coords_100 = spdb.get_all_coordination_numbers_on_facet('100')

sites_111 = list(spdb.get_all_coordination_numbers_on_facet('111').keys())
list_of_sites = []
for item in itertools.combinations_with_replacement(sites_111, 2):
    list_of_sites.append(item)
list_of_sites

[('111_fcc_hollow', '111_fcc_hollow'),
 ('111_fcc_hollow', '111_hcp_hollow'),
 ('111_fcc_hollow', '111_bridge'),
 ('111_fcc_hollow', '111_top'),
 ('111_hcp_hollow', '111_hcp_hollow'),
 ('111_hcp_hollow', '111_bridge'),
 ('111_hcp_hollow', '111_top'),
 ('111_bridge', '111_bridge'),
 ('111_bridge', '111_top'),
 ('111_top', '111_top')]

In [4]:
# get the coordination numbers for the site

# see which entries still need "site" defined

In [5]:
#load the thermodynamics database
thermoDatabase = ThermoDatabase()
thermoDatabase.load(path="/Users/blais.ch/Documents/_01_code/RMG_env_1/RMG-database/input/thermo")

In [6]:
search_str = ""
show_no_site = True
for key in sorted(thermoDatabase.libraries["surfaceThermoPt111"].entries.keys()):
    if show_no_site and len(thermoDatabase.libraries["surfaceThermoPt111"].entries[key].site)>0:
        continue
    else:
        if len(search_str) > 0 and search_str.lower() in key.lower():
            print(f"\'{key}\': \'{thermoDatabase.libraries['surfaceThermoPt111'].entries[key].site}\'")
        elif len(search_str) == 0: 
            print(f"\'{key}\': \'{thermoDatabase.libraries['surfaceThermoPt111'].entries[key].site}\'")

'CH2XCCH3': ''
'CH2XCOH': ''
'CH3NH2X': ''
'CH3OCH2OHX': ''
'CH3XCHXCH2': ''
'CHCCH3X': ''
'CHCHX': ''
'CO2X': ''
'H2C(XO)OCH3': ''
'H2C(XO)XO': ''
'H2NOHX': ''
'HC(O)XO': ''
'HCNX': ''
'HCO2CH3X': ''
'HNOX': ''
'HOOHX': ''
'NH2NH2X': ''
'NH3X': ''
'NHNHX': ''
'NNX': ''
'XCCH2CH3': ''
'XCCH2XC': ''
'XCCH2XCH2': ''
'XCCHCH2': ''
'XCCHXCH2': ''
'XCH2CH2OH': ''
'XCH2CH2XCH2': ''
'XCH2CHCH2': ''
'XCH2NH2': ''
'XCH2XCH': ''
'XCH2XCOH': ''
'XCH2XN': ''
'XCH2XNH': ''
'XCH2XO': ''
'XCHCCH2': ''
'XCHCH2CH3': ''
'XCHCH2XC': ''
'XCHCH2XCH': ''
'XCHCH2XCH2': ''
'XCHCHCH2': ''
'XCHCHO': ''
'XCHCHXCH2': ''
'XCHCHXO': ''
'XCHCO': ''
'XCHCXC': ''
'XCHCXCH': ''
'XCHNH': ''
'XCHNH2': ''
'XCHO': ''
'XCHXCCH3': ''
'XCHXCHCH3': ''
'XCHXCO': ''
'XCHXCXC': ''
'XCHXCXCH': ''
'XCHXN': ''
'XCHXNH': ''
'XCHXO': ''
'XCN': ''
'XCNH': ''
'XCNH2': ''
'XCO': ''
'XCOH': ''
'XCOOH': ''
'XCXCCH3': ''
'XCXCH2': ''
'XCXCHCH3': ''
'XCXCO': ''
'XNCH2': ''
'XNCH3': ''
'XNHCH3': ''
'XNHNH2': ''
'XNHOH': ''
'XNHXN': ''
'XNHXNH

$\theta = E_{ad} -0.1*\frac{X_{m}-X}{X_{m}+1}*\psi-0.2*\frac{X+1}{X_{m}+1}*\overline{CN}$

In [7]:
from rmgpy.quantity import ScalarQuantity

# move this elsewhere when we import routing, should only check once
surface_libraries = []
for lib in thermoDatabase.libraries.keys():
    if "surfaceThermo" in lib: 
        surface_libraries.append(lib)


def correct_binding_energies_extended(thermo, species, metal_to_scale_from=None,
                                       metal_to_scale_to=None, facet_to_scale_from=None, facet_to_scale_to=None,
                                       metal1_site=None,
                                       return_metal1_props = False):
    """
    $\theta = E_{ad} -0.1*\frac{X_{m}-X}{X_{m}+1}*\psi-0.2*\frac{X+1}{X_{m}+1}*\overline{CN}$

    """

    if metal_to_scale_from == metal_to_scale_to and facet_to_scale_from == facet_to_scale_to:
        return thermo
    elif metal_to_scale_from is None or metal_to_scale_to is None or facet_to_scale_from is None or facet_to_scale_to is None:
        raise ValueError("If you are scaling, you must specify both the metal and the facet to scale from and to.")
    
    # get the bond orders for the species
    max_bond_orders, bond_orders, bound_atoms = thermoDatabase.get_bond_order(species, return_bond_orders=True)

    # check for vdw
    if len(bound_atoms) == 0: 
        print(f"species {species.label} has no sites")
        return thermo

    # get the coordination numbers for every site on the facet of interest
    cn_theta1_dict = spdb.get_all_coordination_numbers_on_facet(facet_to_scale_from)
    cn_theta2_dict = spdb.get_all_coordination_numbers_on_facet(facet_to_scale_to)
    psi1 = mpdb.get_psi(metal_to_scale_from)
    psi2 = mpdb.get_psi(metal_to_scale_to)

    metal1_str = metal_to_scale_from + facet_to_scale_from
    metal2_str = metal_to_scale_to + facet_to_scale_to
    print(f"scaling species {species.label} on {metal1_str} to {metal2_str}")

    if metal1_site:
        #check if species has binding site(s) specified. 
        if isinstance(metal1_site, str): 
            sites = {1:(bound_atoms[0],metal1_site)}
        elif isinstance(metal1_site, dict): 
            print(f"species {species.label} has multiple sites")
            sites = metal1_site

    else: 
        # if not, we will have to cycle through all and get lowest energy one. 
        sites = {num:(atom,None) for (num,atom) in enumerate(bound_atoms)}
        print(f"species {species.label} has no site attribute, using {sitetype}")
            
    thetas = {}
    E_ads_new = {}

    
    # if we know the site, we can use that. if we don't , iterate over all of them
    for _, (bound_atom, site) in sites: 
        print(site.label)
        # if site is specified use that
        if site: 
            sitetype = site.label
        else:
            # if site is not specified, choose the site based off of number of atoms adatom bound to
            sitetype = cn_theta1_dict[bound_atom]


        X = bond_orders[bound_atom]
        X_m = max_bond_orders[bound_atom]
        E_ad = (thermo.H298.value_si)/9.68e4 # the paper uses hads
        thetas[sitetype] = E_ad-0.1*(X_m-X)/(X_m+1)*psi1-0.2*(X+1)/(X_m+1)*cn

    # use all of them if species is totally unknown. 
    if sitetype is None:
        # iterate through sites on new surface and pick lowest energy one
        for sitetype1, theta in thetas.items():
            for sitetype2, cn in cn_theta2_dict.items(): 
                E_ads_new[sitetype1+sitetype2] = 0.1*(X_m-X)/(X_m+1)*psi2+0.2*(X+1)/(X_m+1)*cn + theta

    # otherwise, if we know the site from the database value, we can use that.
    else: 
        # Keep the min energy theta? 
        theta_min_site = min(thetas, key=thetas.get)
        theta_min = thetas[theta_min_site]

        # iterate through sites on new surface and 
        for sitetype2, cn in cn_theta2_dict.items(): 
            E_ads_new[sitetype2] = 0.1*(X_m-X)/(X_m+1)*psi2+0.2*(X+1)/(X_m+1)*cn + theta_min

    # get the lowest energy site
    min_h298_key = min(E_ads_new, key=E_ads_new.get)
    min_h298 = ScalarQuantity(E_ads_new[min_h298_key]*9.68e4, "J/mol")
    species.thermo.H298 = min_h298

    # print(f"species {species.label} has been scaled from {sitetype}, h298 of {E_ad} to {min_h298.value_si/9} with a theta of {E_ads_new[min_h298_key]} and a site of {min_h298_key}")
    return E_ads_new, thetas, X_m, X

In [15]:
# test the function. load model each time 
folder = './test_mech/chemkin/'

chemkin_path = os.path.join(folder, 'chem_annotated-gas.inp')
chemkin_surf_path = os.path.join(folder, 'chem_annotated-surface.inp')
dictionary_path = os.path.join(folder, 'species_dictionary.txt')

# load_chemkin_file
species, reactions = load_chemkin_file(chemkin_surf_path, dictionary_path)
# identify a surface species to use for testing
surf_species = species[15]
surf_species.generate_resonance_structures()
display(surf_species)
spec_thermo = surf_species.thermo.to_thermo_data()
surf_species.thermo = spec_thermo

thermo_list = thermoDatabase.get_all_thermo_data(surf_species)
site0 = thermo_list[0][2].site

# new_E_ads, thetas, X_m, X  = correct_binding_energies_extended(spec_thermo, surf_species, metal_to_scale_from='Pt',
#                                        metal_to_scale_to='Cu', facet_to_scale_from='111', facet_to_scale_to='211', metal1_site=site0)

In [ ]:
1

In [ ]:
surf_species

minimum theta is not the correct site. we should probably just go based off of bond order? maybe that would be better. 
- problem is, even if we just use bond order, there are exceptions (O bonds to hollow, not bridge, despire being a 'double bond')
- what we could do it see what error is indroduced if we get bond order wrong. 
- if we know the bond order on one site for certain, should we just apply it to the new site? 

In [ ]:
for num, spec in enumerate(species):
    print(f"species {num}")
    display(spec)
        

NameError: name 'species' is not defined

## validate scaling. 
first, check that all structures don't get set to the hollow configuration

In [52]:
# at the very least, we can try to predict o and oh on 211, and all of the 
# atomic binding energies, as a sanity check. 
import matplotlib.pyplot as plt


# make a dummy scaling method. use psi and theta from above and check against different cns
cns = np.linspace(5, 9, 10)
0.1*(X_m-X)/(X_m+1)*psi2+0.2*(X+1)/(X_m+1)*cn + theta_min




{'111_fcc_hollow': 0.5572916986985423,
 '111_hcp_hollow': 0.6444916986985425,
 '111_bridge': 0.6172916986985424,
 '111_top': 0.6444916986985425}

In [100]:
list1 = ['c', 'o', 'n']
dict1 = {num+1:(atom, None) for num, atom in enumerate(list1)}

In [101]:
dict1

{1: ('c', None), 2: ('o', None), 3: ('n', None)}

## validation of scaled thermo values

In [14]:
# spec we want to validate 
test_spec_labels = ['HX', 'CHX', 'C=[Pt]','C[Pt]', 'CX', 'OX', 'HOX']

# test some bidentates

test_spec = []
for idx, spec in enumerate(species): 
    # if spec.label in test_spec_labels: 
    #     test_spec.append(spec)
    print(idx, spec.label)
    display(spec)

NameError: name 'species' is not defined

In [12]:
species[29]

NameError: name 'species' is not defined

In [21]:
# Get minimum energy site for ch3, ch2, ch and c
surf_species = species[16]
surf_species.generate_resonance_structures()
display(surf_species)
spec_thermo = surf_species.thermo.to_thermo_data()
surf_species.thermo = spec_thermo

{'111_fcc_hollow': -0.7089275269829081,
 '111_hcp_hollow': -0.796127526982908,
 '111_bridge': -0.7689275269829081,
 '111_top': -0.796127526982908}

In [12]:
max_bond_orders, bond_orders, sites = thermoDatabase.get_bond_order(surf_species, return_bond_orders=True)

In [13]:
dir(surf_species)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_molecular_weight',
 '_repr_png_',
 'aug_inchi',
 'calculate_cp0',
 'calculate_cpinf',
 'conformer',
 'contains_surface_site',
 'copy',
 'creation_iteration',
 'energy_transfer_model',
 'explicitly_allowed',
 'fingerprint',
 'from_adjacency_list',
 'from_smiles',
 'generate_aug_inchi',
 'generate_energy_transfer_model',
 'generate_resonance_structures',
 'generate_statmech',
 'generate_transport_data',
 'get_augmented_inchi',
 'get_density_of_states',
 'get_enthalpy',
 'get_entropy',
 'get_free_energy',
 'get_heat_capacity',
 'get_henry_law_constant_data',
 'get_liquid_volumetric_mass_transfer_coefficient_data',
 'get_partition_function',
 'get_

In [14]:
dir(thermoDatabase.libraries['surfaceThermoPt111'].entries[key])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'beta',
 'binding_energies',
 'children',
 'coordination_number',
 'data',
 'data_count',
 'electronegativity',
 'facet',
 'get_all_descendants',
 'index',
 'item',
 'label',
 'long_desc',
 'metal',
 'nodal_distance',
 'parent',
 'psi',
 'rank',
 'reference',
 'reference_type',
 'short_desc',
 'site',
 'surface_site_density',
 'valence_electrons']

In [9]:
sv = mpdb.get_valence_electrons('Sc')
en = mpdb.get_electronegativity('Sc')
beta = mpdb.get_beta('Sc')

psi = (sv**2)/(en**beta)
psi

6.617647058823529